In [1]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd

In [2]:
# load data

df=pd.read_csv('Annthyroid_02_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.026,0.139,0.090,0.153,0


In [3]:
df.shape

(6802, 22)

In [4]:
# data to values
df = df.values

In [5]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Annthyroid.json")
gc = GCForest(config)

In [6]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/home/kader/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# number of class
len(np.unique(y_train))

2

In [8]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-05-15 09:29:07,560][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 21)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 21)],y_test.shape=(2245,)
[ 2018-05-15 09:29:07,564][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-05-15 09:29:07,567][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 09:29:07,568][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-05-15 09:29:07,571][cascade_classifier.fit_transform] X_train.shape=(4557, 21),X_test.shape=(2245, 21)
[ 2018-05-15 09:29:07,575][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 21), X_cur_test.shape=(2245, 21)
[ 2018-05-15 09:29:08,626][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-05-15 09:29:09,428][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.12%
[ 2018-05-15 09:29:10,163][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [9]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 09:29:15,473][cascade_classifier.transform] X_groups_test.shape=[(2245, 21)]
[ 2018-05-15 09:29:15,475][cascade_classifier.transform] group_dims=[21]
[ 2018-05-15 09:29:15,476][cascade_classifier.transform] X_test.shape=(2245, 21)
[ 2018-05-15 09:29:15,479][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 21)


Test Accuracy of ExtraTrees = 98.752784 %


In [10]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[2193    4]
 [  24   24]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      2197
        1.0       0.86      0.50      0.63        48

avg / total       0.99      0.99      0.99      2245



In [11]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-05-15 09:29:16,783][cascade_classifier.transform] X_groups_test.shape=[(2245, 21)]
[ 2018-05-15 09:29:16,785][cascade_classifier.transform] group_dims=[21]
[ 2018-05-15 09:29:16,786][cascade_classifier.transform] X_test.shape=(2245, 21)
[ 2018-05-15 09:29:16,788][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 21)


In [12]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-05-15 09:29:17,968][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 23)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 23)],y_test.shape=(2245,)
[ 2018-05-15 09:29:17,971][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-05-15 09:29:17,972][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 09:29:17,974][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-05-15 09:29:17,975][cascade_classifier.fit_transform] X_train.shape=(4557, 23),X_test.shape=(2245, 23)
[ 2018-05-15 09:29:17,977][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 23), X_cur_test.shape=(2245, 23)
[ 2018-05-15 09:29:18,598][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-05-15 09:29:19,331][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.78%
[ 2018-05-15 09:29:20,023][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [13]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-05-15 09:29:25,380][cascade_classifier.transform] X_groups_test.shape=[(2245, 23)]
[ 2018-05-15 09:29:25,382][cascade_classifier.transform] group_dims=[23]
[ 2018-05-15 09:29:25,383][cascade_classifier.transform] X_test.shape=(2245, 23)
[ 2018-05-15 09:29:25,384][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 23)


Test Accuracy of ExtraTrees = 99.153675 %


In [14]:
# Matrix de confusion
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
print('\n')
print(classification_report(y_test,y_pred))

[[2189    8]
 [  11   37]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      2197
        1.0       0.82      0.77      0.80        48

avg / total       0.99      0.99      0.99      2245



In [15]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')
    
    # Matrix de confusion
    from sklearn.metrics import classification_report, confusion_matrix
    print(confusion_matrix(y_test,y_pred))
    print('\n')
    print(classification_report(y_test,y_pred))

In [16]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-05-15 09:29:26,879][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 23)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 23)],y_test.shape=(2245,)
[ 2018-05-15 09:29:26,881][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-05-15 09:29:26,882][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-05-15 09:29:26,883][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-05-15 09:29:26,884][cascade_classifier.fit_transform] X_train.shape=(4557, 23),X_test.shape=(2245, 23)
[ 2018-05-15 09:29:26,886][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 23), X_cur_test.shape=(2245, 23)
[ 2018-05-15 09:29:27,502][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-05-15 09:29:28,194][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.56%
[ 2018-05-15 09:29:28,935][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 99.153675 %', 0, 'fois')
[[2192    5]
 [  14   34]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      2197
        1.0       0.87      0.71      0.78        48

avg / total       0.99      0.99      0.99      2245



[ 2018-05-15 09:29:36,134][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-05-15 09:29:36,877][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.78%
[ 2018-05-15 09:29:37,585][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.56%
[ 2018-05-15 09:29:38,325][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-05-15 09:29:39,048][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.34%
[ 2018-05-15 09:29:39,761][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.34%
[ 2018-05-15 09:29:40,558][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.34%
[ 2018-05-15 09:29:41,350][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.34%


('Test Accuracy of ExtraTrees = 99.064588 %', 1, 'fois')
[[2190    7]
 [  14   34]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      2197
        1.0       0.83      0.71      0.76        48

avg / total       0.99      0.99      0.99      2245



[ 2018-05-15 09:29:44,735][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-05-15 09:29:45,467][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-05-15 09:29:46,182][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.78%
[ 2018-05-15 09:29:46,918][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-05-15 09:29:47,628][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.34%
[ 2018-05-15 09:29:48,369][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.56%
[ 2018-05-15 09:29:49,099][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.34%
[ 2018-05-15 09:29:49,827][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.34%


...

('Test Accuracy of ExtraTrees = 98.886414 %', 17, 'fois')
[[2191    6]
 [  19   29]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99      2197
        1.0       0.83      0.60      0.70        48

avg / total       0.99      0.99      0.99      2245



[ 2018-05-15 09:32:01,700][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-05-15 09:32:02,421][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-05-15 09:32:03,116][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.12%
[ 2018-05-15 09:32:03,833][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-05-15 09:32:04,557][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.56%
[ 2018-05-15 09:32:05,398][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.34%
[ 2018-05-15 09:32:06,144][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.78%
[ 2018-05-15 09:32:06,899][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.90%


('Test Accuracy of ExtraTrees = 99.064588 %', 18, 'fois')
[[2193    4]
 [  17   31]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      2197
        1.0       0.89      0.65      0.75        48

avg / total       0.99      0.99      0.99      2245



[ 2018-05-15 09:32:10,395][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-05-15 09:32:11,136][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-05-15 09:32:11,843][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.34%
[ 2018-05-15 09:32:12,580][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.34%
[ 2018-05-15 09:32:13,328][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=99.56%
[ 2018-05-15 09:32:14,079][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=99.12%
[ 2018-05-15 09:32:14,828][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=99.34%
[ 2018-05-15 09:32:15,535][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.68%


('Test Accuracy of ExtraTrees = 99.020045 %', 19, 'fois')
[[2192    5]
 [  17   31]]


             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      2197
        1.0       0.86      0.65      0.74        48

avg / total       0.99      0.99      0.99      2245



# ('Test Accuracy of ExtraTrees = 99.287305 %', 2, 'fois')